In [ ]:
from genet import predict as prd

In [ ]:
seq_wt   = 'ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGAAGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT'
seq_ed   = 'ATGACAATAAAAGACAACACCCTTGCCTTGTGGAGTTTTCAAAGCTCCCAGAAACTGAGACGAACTATAACCTGCAAATGTCAACTGAAACCTTAAAGTGAGTATTTAATTGAGCTGAAGT'
alt_type = 'sub1'

df_pe = prd.pe_score(seq_wt, seq_ed, alt_type)
df_pe.head()

In [ ]:
df_test = df_pe['Edited74_On'].str.replace('x', '')
df_test

In [ ]:
from Bio.Seq import transcribe, reverse_complement, reverse_complement_rna

In [ ]:
list_rtpbs = [reverse_complement(seq.replace('x', '')) for seq in df_test]
list_rtpbs

In [ ]:
selected = ['WT74_On', 'PBSlen', 'RTlen', 'RT-PBSlen', 'Edit_pos', 'Edit_len', 'RHA_len', 'PE2max_score']
df_pe_summary = df_pe[selected]
df_pe_summary.insert(1, 'RT-PBSseq', list_rtpbs)

In [ ]:
df_pe_summary

In [ ]:
# genet ToDo

# 1. output dataframe 간단하게 표현하기
# 2. GetGene 불러와서 Base editing용 gRNA 디자인해주기

In [ ]:
class DeepPrime:
    '''
    DeepPrime: pegRNA activity prediction models
    

    
    '''
    def __init__(self,
                Ref_seq: str, 
                ED_seq: str, 
                sAlt: str,
                sID:str       = 'Sample',
                pe_system:str = 'PE2max',
                cell_type:str = 'HEK293T',
                pbs_min:int   = 7,
                pbs_max:int   = 15,
                rtt_max:int   = 40
                ):
        print('\nDeepPrime: pegRNA activity prediction models')
        print('ID: %s' % sID)
        print('Refseq:', Ref_seq)
        print('EDseq :', ED_seq)

        self.Ref_seq   = Ref_seq
        self.ED_seq    = ED_seq
        self.sAlt      = sAlt
        self.sID       = sID
        self.pe_system = pe_system
        self.cell_type = cell_type
        self.pbs_min   = pbs_min
        self.pbs_max   = pbs_max
        self.rtt_max   = rtt_max




    def pe_score(self):
        print('start pe_scre', self.Ref_seq, self.ED_seq, self.sAlt, )



In [ ]:
test_class = DeepPrime('test', 'test', 'test')
# test_class